In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
!pip install konlpy pandas seaborn gensim wordcloud python-mecab-ko wget svgling

In [6]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-nanum is already the newest version (20200506-1).
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 12 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/usr/share/fonts/truetype: skipping, looped directory detected
/usr/share/fonts/truetype/humor-sans: skipping, looped directory detected
/usr/share/fonts/truetype/liberation: skipping, looped directory detected
/usr/share/fonts/truetype/

In [34]:
!pip uninstall keras

Found existing installation: keras 2.12.0
Uninstalling keras-2.12.0:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/keras-2.12.0.dist-info/*
    /usr/local/lib/python3.10/dist-packages/keras/*
Proceed (Y/n)? y
  Successfully uninstalled keras-2.12.0


In [35]:
!pip uninstall tensorflow
!pip install tensorflow==2.12.0

Found existing installation: tensorflow 2.12.0
Uninstalling tensorflow-2.12.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.10/dist-packages/tensorflow-2.12.0.dist-info/*
    /usr/local/lib/python3.10/dist-packages/tensorflow/*
Proceed (Y/n)? y
  Successfully uninstalled tensorflow-2.12.0
  Using cached tensorflow-2.12.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (585.9 MB)
  Using cached keras-2.12.0-py2.py3-none-any.whl (1.7 MB)


In [125]:
import re
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV

import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.font_manager as fm
from wordcloud import WordCloud

import nltk
from mecab import MeCab

import wget, os

In [126]:
FONT_PATH = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
font_name = fm.FontProperties(fname=FONT_PATH, size=10).get_name()
print(font_name)
plt.rcParams['font.family']=font_name
assert plt.rcParams['font.family'] == [font_name], "한글 폰트가 설정되지 않았습니다."

NanumGothic


## 데이터 전처리

In [127]:
nltk.download('stopwords')
mecab = MeCab()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [128]:
def mecab_tokenizer(string):
    return list("/".join(res) for res in mecab.pos(str(string)))

In [129]:
PATH = '/content/drive/MyDrive/미프4_2'

train_df = pd.read_csv(os.path.join(PATH, 'train.csv'))
test_df = pd.read_csv(os.path.join(PATH, 'test.csv'))
# test_df = test_df.rename(columns=columns_dict)
print(train_df.shape)
print(test_df.shape)

(3706, 2)
(929, 2)


In [130]:
label_dict = {
    '코드1': 0,
    '코드2': 0,
    '웹': 1,
    '이론': 2,
    '시스템 운영': 3,
    '원격': 4
}
preprocessed_df = train_df.replace({'label': label_dict}).copy()

In [131]:
test_df.head()

,id,text
0,0,팀즈는 노트북으로 접속하고 강의는 데스크톱 이용하려고 하는데 문제는 없을까요? 이...
1,1,"셀프테스트에서 받은 점수가 해당 교육을 이수하고, 취업 연계등을 하는데 있어서 영향..."
2,2,a= int(input('정수 A의 값을 입력하시오.:'))\nb= int(inpu...
3,3,"def max4(a,b,c,d):\n maximum = a\n if b&..."
4,4,i**=2 가 i **=2 와 왜 같은지가 이해가 잘 안됩니다!!


In [132]:
X_tr,X_val,y_train,y_valid = train_test_split(preprocessed_df.text,preprocessed_df.label, test_size=0.25,random_state=42)
X_te = test_df.text

In [133]:
np.shape(X_tr), np.shape(X_val), np.shape(X_te)

((2779,), (927,), (929,))

In [150]:
import re
import string

removal_list =  "‘’◇‘”’'·\“·△●■()\">>`/-∼=ㆍ<>.?!【】…◆%"
removal_list += string.punctuation
def cleansing_special(sentence: str = None) -> str:
    """
    특수문자를 전처리를 하는 함수
    :param sentence: 전처리 대상 문장
    :return: 전처리 완료된 문장
    """
    #sentence = re.sub("[.,\'\"’‘”“!?]", "", sentence)
    sentence = re.sub("[^가-힣0-9a-zA-Z\\s]", " ", sentence)
    sentence = re.sub("\s+", " ", sentence)

    sentence = sentence.translate(str.maketrans(removal_list, ' '*len(removal_list)))
    sentence = sentence.strip()
    return sentence

In [151]:
X_tr = X_tr.apply(cleansing_special)
X_val = X_val.apply(cleansing_special)
X_te = X_te.apply(cleansing_special)

In [152]:
np.shape(X_tr), np.shape(X_val), np.shape(X_te)

((2779,), (927,), (929,))

## 2. N-gram 토큰화

In [153]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [154]:
# N-grams
count_vectorizer = CountVectorizer()
count_mecab_vectorizer = CountVectorizer(tokenizer=mecab_tokenizer)

In [155]:
x_tr_counts = count_vectorizer.fit_transform(X_tr)
x_val_counts = count_vectorizer.transform(X_val)
x_te_counts = count_vectorizer.transform(X_te)

In [156]:
x_tr_mecab_counts = count_mecab_vectorizer.fit_transform(X_tr)
x_val_mecab_counts = count_mecab_vectorizer.transform(X_val)
x_te_mecab_counts = count_mecab_vectorizer.transform(X_te)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [157]:
transformer = TfidfTransformer()
x_tr_tfidf = transformer.fit_transform(x_tr_mecab_counts)
x_val_tfidf = transformer.transform(x_val_mecab_counts)
x_te_tfidf = transformer.transform(x_te_mecab_counts)

In [158]:
tfidf_vectorizer = TfidfVectorizer(tokenizer=mecab_tokenizer)
x_tr_tfidfv = tfidf_vectorizer.fit_transform(X_tr)
x_val_tfidfv = tfidf_vectorizer.transform(X_val)
x_te_tfidfv = tfidf_vectorizer.transform(X_te)

In [159]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

In [107]:
!pip install tensorflow-addons

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 612.3/612.3 kB 5.3 MB/s eta 0:00:00


In [108]:
import tensorflow_addons as tfa

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [160]:
def get_last_layer_units_and_activation(num_classes):
    """Gets the # units and activation function for the last network layer.

    # Arguments
        num_classes: int, number of classes.

    # Returns
        units, activation values.
    """
    if num_classes == 2:
        activation = 'sigmoid'
        units = 1
    else:
        activation = 'softmax'
        units = num_classes
    return units, activation

In [161]:
import tensorflow as tf
from tensorflow.keras import models
from tensorflow.keras.layers import Dense, Dropout

def mlp_model(layers, units, dropout_rate, input_shape, num_classes):
    """Creates an instance of a multi-layer perceptron model.

    # Arguments
        layers: int, number of `Dense` layers in the model.
        units: int, output dimension of the layers.
        dropout_rate: float, percentage of input to drop at Dropout layers.
        input_shape: tuple, shape of input to the model.
        num_classes: int, number of output classes.

    # Returns
        An MLP model instance.
    """
    op_units, op_activation = get_last_layer_units_and_activation(num_classes)
    model = models.Sequential()
    model.add(Dropout(rate=dropout_rate, input_shape=input_shape))

    for _ in range(layers-1):
        model.add(Dense(units=units, activation='relu'))
        model.add(Dropout(rate=dropout_rate))

    model.add(Dense(units=op_units, activation=op_activation))
    return model

In [163]:
def train_ngram_model(x_train, x_val, train_labels, val_labels,
                      learning_rate=1e-3,
                      epochs=1000,
                      batch_size=128,
                      layers=2,
                      units=64,
                      dropout_rate=0.2):
    """Trains n-gram model on the given dataset.

    # Arguments
        data: tuples of training and test texts and labels.
        learning_rate: float, learning rate for training model.
        epochs: int, number of epochs.
        batch_size: int, number of samples per batch.
        layers: int, number of `Dense` layers in the model.
        units: int, output dimension of Dense layers in the model.
        dropout_rate: float: percentage of input to drop at Dropout layers.

    # Raises
        ValueError: If validation data has label values which were not seen
            in the training data.
    """


    # Verify that validation labels are in the same range as training labels.
    num_classes = len(np.unique(val_labels))
    unexpected_labels = [v for v in val_labels if v not in range(num_classes)]

    if len(unexpected_labels):
        raise ValueError('Unexpected label values found in the validation set:'
                         ' {unexpected_labels}. Please make sure that the '
                         'labels in the validation set are in the same range '
                         'as training labels.'.format(
                             unexpected_labels=unexpected_labels))

    # Create model instance.
    model = mlp_model(layers=layers,
                      units=units,
                      dropout_rate=dropout_rate,
                      input_shape=x_train.shape[1:],
                      num_classes=num_classes)

    # Compile model with learning parameters.
    if num_classes == 2:
        loss = 'binary_crossentropy'
    else:
        loss = 'sparse_categorical_crossentropy'
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss=loss,  metrics=['acc'])

    # Create callback for early stopping on validation loss. If the loss does
    # not decrease in two consecutive tries, stop training.
    callbacks = [tf.keras.callbacks.EarlyStopping(
        monitor='val_loss', patience=2)]

    # Train and validate model.
    history = model.fit(
            x_train,
            train_labels,
            epochs=epochs,
            callbacks=callbacks,
            validation_data=(x_val, val_labels),
            verbose=2,  # Logs once per epoch.
            batch_size=batch_size)

    # Print results.
    history = history.history
    print('Validation accuracy: {acc}, loss: {loss}'.format(
            acc=history['val_acc'][-1], loss=history['val_loss'][-1]))

    return model

In [164]:
x_train = x_tr_tfidf.toarray()
x_valid = x_val_tfidf.toarray()
x_test = x_te_tfidf.toarray()

In [165]:
from imblearn.over_sampling import SMOTE

smote =SMOTE(random_state=42)
x_train_smt, y_train_smt = smote.fit_resample(x_train, y_train)

In [111]:
np.shape(x_train), np.shape(y_train)

((2779, 9917), (2779,))

In [114]:
np.shape(x_valid), np.shape(y_valid)

((927, 9917), (927,))

In [166]:
# 모델 학습
model1 = train_ngram_model(x_train, x_valid, y_train, y_valid)

Epoch 1/1000
22/22 - 2s - loss: 1.5498 - acc: 0.4106 - val_loss: 1.4642 - val_acc: 0.4315 - 2s/epoch - 70ms/step
Epoch 2/1000
22/22 - 1s - loss: 1.3681 - acc: 0.4494 - val_loss: 1.2775 - val_acc: 0.4509 - 705ms/epoch - 32ms/step
Epoch 3/1000
22/22 - 1s - loss: 1.1675 - acc: 0.5286 - val_loss: 1.1009 - val_acc: 0.5351 - 691ms/epoch - 31ms/step
Epoch 4/1000
22/22 - 1s - loss: 0.9694 - acc: 0.6913 - val_loss: 0.9434 - val_acc: 0.6980 - 690ms/epoch - 31ms/step
Epoch 5/1000
22/22 - 1s - loss: 0.7965 - acc: 0.8305 - val_loss: 0.8131 - val_acc: 0.7735 - 690ms/epoch - 31ms/step
Epoch 6/1000
22/22 - 1s - loss: 0.6537 - acc: 0.8762 - val_loss: 0.7140 - val_acc: 0.8004 - 694ms/epoch - 32ms/step
Epoch 7/1000
22/22 - 1s - loss: 0.5367 - acc: 0.9036 - val_loss: 0.6407 - val_acc: 0.8231 - 682ms/epoch - 31ms/step
Epoch 8/1000
22/22 - 1s - loss: 0.4531 - acc: 0.9108 - val_loss: 0.5908 - val_acc: 0.8231 - 690ms/epoch - 31ms/step
Epoch 9/1000
22/22 - 1s - loss: 0.3888 - acc: 0.9234 - val_loss: 0.5512 - v

In [167]:
mode2 =  train_ngram_model(x_train_smt, x_valid, y_train_smt, y_valid)

Epoch 1/1000
47/47 - 2s - loss: 1.4581 - acc: 0.7303 - val_loss: 1.3020 - val_acc: 0.8339 - 2s/epoch - 47ms/step
Epoch 2/1000
47/47 - 1s - loss: 0.9803 - acc: 0.9262 - val_loss: 0.9134 - val_acc: 0.8447 - 1s/epoch - 29ms/step
Epoch 3/1000
47/47 - 1s - loss: 0.5836 - acc: 0.9386 - val_loss: 0.6663 - val_acc: 0.8490 - 1s/epoch - 29ms/step
Epoch 4/1000
47/47 - 1s - loss: 0.3729 - acc: 0.9500 - val_loss: 0.5449 - val_acc: 0.8544 - 1s/epoch - 29ms/step
Epoch 5/1000
47/47 - 2s - loss: 0.2573 - acc: 0.9665 - val_loss: 0.4862 - val_acc: 0.8554 - 2s/epoch - 38ms/step
Epoch 6/1000
47/47 - 3s - loss: 0.1934 - acc: 0.9750 - val_loss: 0.4565 - val_acc: 0.8511 - 3s/epoch - 73ms/step
Epoch 7/1000
47/47 - 3s - loss: 0.1510 - acc: 0.9799 - val_loss: 0.4411 - val_acc: 0.8554 - 3s/epoch - 55ms/step
Epoch 8/1000
47/47 - 2s - loss: 0.1263 - acc: 0.9816 - val_loss: 0.4340 - val_acc: 0.8522 - 2s/epoch - 45ms/step
Epoch 9/1000
47/47 - 1s - loss: 0.1045 - acc: 0.9846 - val_loss: 0.4313 - val_acc: 0.8479 - 1s/e

In [171]:
result = mode2.predict(x_test)
result

30/30 [==============================] - 0s 7ms/step


array([[2.7136834e-02, 1.8941421e-02, 1.8109672e-02, 9.2325616e-01,
        1.2555791e-02],
       [7.2564639e-02, 9.8608665e-02, 1.8796927e-01, 6.3160527e-01,
        9.2520835e-03],
       [9.9833494e-01, 7.5084704e-04, 4.1611411e-04, 1.8397754e-04,
        3.1416173e-04],
       ...,
       [3.6475759e-03, 3.7110690e-03, 1.5998874e-02, 9.7506964e-01,
        1.5727868e-03],
       [2.0592054e-02, 9.4648182e-01, 2.8286526e-02, 3.2881319e-03,
        1.3513443e-03],
       [9.0661651e-01, 8.1132784e-02, 6.5767555e-04, 1.0384540e-02,
        1.2084104e-03]], dtype=float32)

In [176]:
np.argmax(result, axis=1)

array([3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 3, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 2, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 2,
       0, 0, 3, 0, 0, 1, 0, 2, 1, 0, 0, 0, 1, 0, 0, 3, 0, 0, 1, 0, 0, 3,
       0, 1, 0, 0, 3, 2, 0, 2, 0, 0, 4, 3, 0, 2, 0, 0, 2, 0, 0, 0, 0, 3,
       3, 0, 0, 3, 2, 0, 0, 0, 2, 0, 0, 0, 1, 3, 0, 3, 3, 0, 0, 0, 0, 0,
       0, 3, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 1, 0, 0, 4, 0, 0, 0, 0, 0, 3,
       3, 3, 0, 0, 0, 0, 1, 1, 0, 0, 3, 4, 0, 0, 0, 0, 2, 0, 0, 0, 0, 3,
       0, 2, 0, 2, 2, 2, 0, 0, 0, 2, 2, 0, 2, 2, 2, 2, 0, 3, 0, 0, 3, 2,
       2, 2, 2, 2, 2, 2, 2, 0, 0, 2, 2, 0, 0, 3, 0, 2, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 3, 3, 2, 3, 0, 2, 2, 3, 3, 0, 2, 2, 0, 2, 3, 2, 2, 0, 3,
       3, 0, 2, 0, 0, 0, 2, 0, 0, 0, 4, 0, 2, 2, 0, 2, 0, 0, 2, 0, 0, 0,
       0, 0, 0, 0, 0, 3, 3, 0, 0, 0, 0, 2, 0, 2, 0, 2, 0, 0, 0, 2, 0, 0,
       0, 2, 0, 0, 0, 0, 0, 2, 0, 0, 1, 1, 0, 0, 4, 3, 2, 0, 0, 0, 0, 0,
       0, 0, 0, 3, 3, 2, 0, 3, 2, 2, 3, 0, 3, 0, 0,

In [178]:
import pandas as pd

# numpy 배열을 pandas DataFrame으로 변환하고 인덱스 이름을 'ID'로 설정
df_result = pd.DataFrame(np.argmax(result, axis=1), columns=['label'])
df_result.index.name = 'ID'

# CSV 파일로 저장
df_result.to_csv('/content/drive/MyDrive/미프4_2/kaggle_result.csv', index=True)
